# Module 3 - Explore your app

## Objectives

* Learn about Kubernetes Pods.
* Learn about Kubernetes Nodes.
* Troubleshoot deployed applications.

## Kubernetes Pods

When you created a Deployment in [Module 2](./3-WKSHP-K8S101-mod2.ipynb), Kubernetes created a **Pod** to host your application instance. A Pod is a Kubernetes abstraction that represents a group of one or more application containers (such as Docker), and some shared resources for those containers. Those resources include:

* Shared storage, as Volumes
* Networking, as a unique cluster IP address
* Information about how to run each container, such as the container image version or specific ports to use

A Pod models an application-specific "logical host" and can contain different application containers which are relatively tightly coupled. For example, a Pod might include both the container with your Node.js app as well as a different container that feeds the data to be published by the Node.js webserver. The containers in a Pod share an IP Address and port space, are always co-located and co-scheduled, and run in a shared context on the same Node.

Pods are the atomic unit on the Kubernetes platform. When we create a Deployment on Kubernetes, that Deployment creates Pods with containers inside them (as opposed to creating containers directly). Each Pod is tied to the Node where it is scheduled, and remains there until termination (according to restart policy) or deletion. In case of a Node failure, identical Pods are scheduled on other available Nodes in the cluster.

>A Pod is a group of one or more application containers (such as Docker) and includes shared storage (volumes), IP address and information about how to run them.

## Pods overview

<img src="https://d33wubrfki0l68.cloudfront.net/fe03f68d8ede9815184852ca2a4fd30325e5d15a/98064/docs/tutorials/kubernetes-basics/public/images/module_03_pods.svg"  width="600" title="test" alt="">

## Nodes

A Pod always runs on a **Node**. A Node is a worker machine in Kubernetes and may be either a virtual or a physical machine, depending on the cluster. Each Node is managed by the Master. A Node can have multiple pods, and the Kubernetes master automatically handles scheduling the pods across the Nodes in the cluster. The Master's automatic scheduling takes into account the available resources on each Node.

Every Kubernetes Node runs at least:

* Kubelet, a process responsible for communication between the Kubernetes Master and the Node; it manages the Pods and the containers running on a machine.
* A container runtime (like Docker) responsible for pulling the container image from a registry, unpacking the container, and running the application.

>Containers should only be scheduled together in a single Pod if they are tightly coupled and need to share resources such as disk.

## Node overview

<img src="https://d33wubrfki0l68.cloudfront.net/5cb72d407cbe2755e581b6de757e0d81760d5b86/a9df9/docs/tutorials/kubernetes-basics/public/images/module_03_nodes.svg"  width="600" title="test" alt="">

## Troubleshooting with kubectl

In [Module 2](./3-WKSHP-K8S101-mod2.ipynb), you used Kubectl command-line interface. You'll continue to use it in Module 3 to get information about deployed applications and their environments. The most common operations can be done with the following kubectl commands:

* **kubectl get** - list resources
* **kubectl describe** - show detailed information about a resource
* **kubectl logs** - print the logs from a container in a pod
* **kubectl exec** - execute a command on a container in a pod

You can use these commands to see when applications were deployed, what their current statuses are, where they are running and what their configurations are.

Now that we know more about our cluster components and the command line, let's explore our application.

>A node is a worker machine in Kubernetes and may be a VM or physical machine, depending on the cluster. Multiple Pods can run on one Node.

### Lab 3 - Explore your app
#### Step 1 of 4

Let’s verify that the application we deployed in the previous scenario is running. We’ll use the `kubectl get` command and look for existing Pods:

In [1]:
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)

kubectl get pods | grep $studentId


kubernetes-bootcamp-student1125-55cc476c66-f7rdf   1/1     Running   0          13d


Next, to view what containers are inside that Pod and what images are used to build those containers we run the `describe pods` command:


In [2]:
POD_NAME=$(kubectl get pods -o go-template --template '{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}' | grep $studentId) 
echo $POD_NAME
kubectl describe pods $POD_NAME

kubernetes-bootcamp-student1125-55cc476c66-f7rdf
Name:         kubernetes-bootcamp-student1125-55cc476c66-f7rdf
Namespace:    k8shacktenant
Priority:     0
Node:         worker28.etc.fr.comm.hpecorp.net/16.31.84.70
Start Time:   Fri, 02 Apr 2021 09:42:22 +0000
Labels:       app=kubernetes-bootcamp-student1125
              pod-template-hash=55cc476c66
Annotations:  cni.projectcalico.org/podIP: 10.192.1.43/32
              cni.projectcalico.org/podIPs: 10.192.1.43/32
              kubernetes.io/psp: hcp-psp-privileged
Status:       Running
IP:           10.192.1.43
IPs:
  IP:           10.192.1.43
Controlled By:  ReplicaSet/kubernetes-bootcamp-student1125-55cc476c66
Containers:
  kubernetes-bootcamp:
    Container ID:   docker://eb87866b873bb85a8829f17627fe4a002a64b85a3deaf1ca91d29fc44b57bf82
    Image:          gcr.io/google-samples/kubernetes-bootcamp:v1
    Image ID:       docker-pullable://gcr.io/google-samples/kubernetes-bootcamp@sha256:0d6b8ee63bb57c5f5b6156f446b3bc3b3c143d233037f

We see here details about the Pod’s container: IP address, the ports used and a list of events related to the lifecycle of the Pod.

The output of the `describe` command is extensive and covers some concepts that we didn’t explain yet, but don’t worry, they will become familiar by the end of this bootcamp.

>Note: the describe command can be used to get detailed information about most of the kubernetes primitives: node, pods, deployments. The describe output is designed to be human readable, not to be scripted against.

#### Step 2 of 4

Recall that Pods are running in an isolated, private network - so we need to proxy access to them so we can debug and interact with them. To do this, we created a port-forward in [Module 2](./3-WKSHP-K8S101-mod2.ipynb) using the `kubectl port-forward` command in a second terminal window. 

To see the output of our application, run can also call the API using a `curl` request.

`curl http://localhost:$port/api/v1/namespaces/default/pods/$POD_NAME/proxy/`

The url is the route to the API of the Pod.

In [3]:
port=$(expr $(echo ${studentId} | tr -cd '[[:digit:]]') + 8000)

echo Name of the Pod: $POD_NAME
echo Port number: $port

curl http://localhost:$port/api/v1/namespaces/default/pods/$POD_NAME/proxy/

Name of the Pod: kubernetes-bootcamp-student1125-55cc476c66-f7rdf
Port number: 9125
Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-student1125-55cc476c66-f7rdf | v=1


#### Step 3 of 4

Anything that the application would normally send to `STDOUT` becomes logs for the container within the Pod. We can retrieve these logs using the `kubectl logs` command:

In [10]:
kubectl logs $POD_NAME

Kubernetes Bootcamp App Started At: 2021-04-02T09:42:25.096Z | Running On:  kubernetes-bootcamp-student1125-55cc476c66-f7rdf 

Running On: kubernetes-bootcamp-student1125-55cc476c66-f7rdf | Total Requests: 1 | App Uptime: 441571.123 seconds | Log Time: 2021-04-07T12:21:56.219Z
Running On: kubernetes-bootcamp-student1125-55cc476c66-f7rdf | Total Requests: 2 | App Uptime: 442086.366 seconds | Log Time: 2021-04-07T12:30:31.462Z
Running On: kubernetes-bootcamp-student1125-55cc476c66-f7rdf | Total Requests: 3 | App Uptime: 442757.864 seconds | Log Time: 2021-04-07T12:41:42.961Z
Running On: kubernetes-bootcamp-student1125-55cc476c66-f7rdf | Total Requests: 4 | App Uptime: 442799.759 seconds | Log Time: 2021-04-07T12:42:24.855Z


>The name of the container itself can be omitted since we only have a single container in the Pod.

#### Step 4 of 4

We can execute commands directly on the container once the Pod is up and running. For this, we use the `exec` command and use the name of the Pod as a parameter. Let’s list the environment variables:

In [5]:
kubectl exec $POD_NAME -- env

PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME=kubernetes-bootcamp-student1125-55cc476c66-f7rdf
KUBERNETES_SERVICE_PORT_HTTPS=443
KUBERNETES_PORT=tcp://10.96.0.1:443
KUBERNETES_PORT_443_TCP=tcp://10.96.0.1:443
HELLO_WORLD_SERVICE_DEMOUSER_PORT_8080_TCP_PROTO=tcp
KUBERNETES_SERVICE_HOST=10.96.0.1
KUBERNETES_PORT_443_TCP_PROTO=tcp
KUBERNETES_PORT_443_TCP_ADDR=10.96.0.1
HELLO_WORLD_SERVICE_DEMOUSER_SERVICE_PORT_HTTP_HELLO=8080
HELLO_WORLD_SERVICE_DEMOUSER_PORT=tcp://10.106.252.243:8080
KUBERNETES_SERVICE_PORT=443
HELLO_WORLD_SERVICE_DEMOUSER_SERVICE_HOST=10.106.252.243
HELLO_WORLD_SERVICE_DEMOUSER_PORT_8080_TCP_ADDR=10.106.252.243
KUBERNETES_PORT_443_TCP_PORT=443
HELLO_WORLD_SERVICE_DEMOUSER_SERVICE_PORT=8080
HELLO_WORLD_SERVICE_DEMOUSER_PORT_8080_TCP=tcp://10.106.252.243:8080
HELLO_WORLD_SERVICE_DEMOUSER_PORT_8080_TCP_PORT=8080
NPM_CONFIG_LOGLEVEL=info
NODE_VERSION=6.3.1
HOME=/root


>Again, worth mentioning that the name of the container itself can be omitted since we only have a single container in the Pod.

Next let’s check the source code of the app in the server.js file:


In [6]:
kubectl exec $POD_NAME -- cat server.js

var http = require('http');
var requests=0;
var podname= process.env.HOSTNAME;
var startTime;
var host;
var handleRequest = function(request, response) {
  response.setHeader('Content-Type', 'text/plain');
  response.writeHead(200);
  response.write("Hello Kubernetes bootcamp! | Running on: ");
  response.write(host);
  response.end(" | v=1\n");
  console.log("Running On:" ,host, "| Total Requests:", ++requests,"| App Uptime:", (new Date() - startTime)/1000 , "seconds", "| Log Time:",new Date());
}
var www = http.createServer(handleRequest);
www.listen(8080,function () {
    startTime = new Date();;
    host = process.env.HOSTNAME;
    console.log ("Kubernetes Bootcamp App Started At:",startTime, "| Running On: " ,host, "\n" );
});


Go back to your ` Terminal ` session and stop the port-forward, using CTRL-C

This terminates module 3. Let's continue with [module 4, Expose your app publicly](./5-WKSHP-K8S101-mod4.ipynb).